In [2]:
# main  code 
# before running code make sure your video should be in same folder 

import cv2 as cv
import numpy as np
import xlwings as xl

frameNo = 0 # considering video is of 30 fps

col = 2 # manage excel cols
vid = cv.VideoCapture('vid.mp4') 

sheet = xl.Book('book.xlsx')
sheet = sheet.sheets['Sheet1']

# guard =  [0, 0, 0, 0] # to manage exit contition of ball {under dev}
tempQuad = [5, 5, 5, 5] # to prevent over writing of records , record only when a ball change its quadrant


sheet['A1'].value = [['Color', 'Quadrant', 'Existance', 'time'], ["", "", "", ""]]#initial entry

#-------------------------------------------------------------------------------------------------------

def sec(seconds):
	seconds = seconds % (24 * 3600)
	hour = seconds // 3600
	seconds %= 3600
	minutes = seconds // 60
	seconds %= 60
	
	return "%d:%02d:%02d" % (hour, minutes, seconds)
	


#-------------------------------------------------------------------------------------------------------


def entry(color="", quadrant="", existanceType="", time="", edit=False):
    global col
    col+=1 
    sheet['A'+str(col)].value = [[color, quadrant, existanceType, time]]
    

#-------------------------------------------------------------------------------------------------------

def getQuadrant(width, height):
    if (width > 386):
        return 1 if height > 167 else 4
    else:
        return 2 if height > 167 else 3
    
#-------------------------------------------------------------------------------------------------------

def getBall(frame, _mask, cname):
   
    global tempQuad
    global guard
    global frameNo
    
    kernal = np.ones((5, 5), "uint8")
    _mask = cv.dilate(_mask, kernal)
    contours, hierarchy = cv.findContours( _mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    for pic, contour in enumerate(contours):

        area = cv.contourArea(contour)

        if cname == "white" :  area -= 200 # to prevent wrong prediction, white blend with background
    
        if cname == 'yellow':
            i = 0
        if cname == 'green':
            i = 1
        if cname == 'orange':
            i = 2
        if cname == 'white':
            i = 3

        if (area > 300):
            x, y, w, h = cv.boundingRect(contour)
            quad = getQuadrant(x, y)

            # guard[i] = 1 #under dev { for exit timings }
            frame = cv.rectangle(frame, (x, y),  (x + w, y + h), (0, 255, 0), 2)
            cv.putText(frame, str(quad)+" "+cname, (x, y),cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0))
            if (tempQuad[i] != quad) : entry(cname, quad, "in",sec(frameNo/30.0)) #30 fps
            tempQuad[i] = quad
        
#-------------------------------------------------------------------------------------------------------

def tracKBall(frame, masks):
    
    global frameNo
    getBall(frame, masks[0], "yellow")
    getBall(frame, masks[1], "green")
    getBall(frame, masks[2], "orange")
    getBall(frame, masks[3], "white")
    frameNo+=1 ;
    cv.imshow("Multiple Color Detection in Real-TIme",cv.cvtColor(frame, cv.COLOR_HSV2BGR))

#-------------------------------------------------------------------------------------------------------

while 1:
    
    get, frame = vid.read()
    frame = cv.resize(frame, (600, 350))
    frame = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    if cv.waitKey(20) == ord('q'):break

    # values are calibrated using trackbar // white color is not calibrated yet
    m1 = cv.inRange(frame, (22, 85, 0), (28, 255, 255))  # yellow
    m2 = cv.inRange(frame, (31, 53, 0), (98, 255, 255))  # green
    m3 = cv.inRange(frame, (3, 67, 164), (15, 246, 255))  # orange
    m4 = cv.inRange(frame, (23, 13, 210) ,(43, 224, 255))  # white

    tracKBall(frame, [m1, m2, m3,m4])

#-------------------------------------------------------------------------------------------------------


vid.release()
cv.destroyAllWindows()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
